<a href="https://colab.research.google.com/github/fpsomad/fpsomad/blob/main/Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# εισαγωγή βιβλιοθηκών
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [19]:
# εισαγωγή συνόλου δεδομένων
data = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/DarkNet.csv')



<ipython-input-19-12fa18687824>:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/DarkNet.csv')


In [20]:
# ταξινόμηση δεδομένων με βάση την κίνηση
data.groupby(data['Label-1']).size()


Label-1
Non-Tor    30931
NonVPN     22740
Tor         1392
VPN        13517
dtype: int64

In [21]:
# ταξινόμηση δεδομένων με βάση την εφαρμογή
data.groupby(data['Label-2']).size()


Label-2
AUDIO-STREAMING     1484
Audio-Streaming     6055
Browsing            5192
Chat               11478
Email               6145
File-Transfer      11098
File-transfer         84
P2P                13711
VOIP                3566
Video-Streaming     9486
Video-streaming      281
dtype: int64